In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# === CONFIGURACIÓN ===
ruta_csv = "M1_30Hz.csv"   # cambia al nombre real de tu archivo
big_endian = False              # True = [byte alto, byte bajo]; False = little endian

# === LECTURA ROBUSTA DEL CSV ===
df = pd.read_csv(
    ruta_csv,
    sep=r'\s*;\s*',   # permite espacios alrededor del ';'
    decimal=',',
    engine='python'
)

# Si el tiempo se leyó como índice, lo reseteamos
df.reset_index(inplace=True)

df = df.drop(columns='SPI (1,2,3,4)')
df = df.rename(columns={'index': 'Time', 'Time': 'spi'})


In [ ]:
muestras = []
for i in range(0, len(df)):
    # Evita índices fuera de rango (por si se truncó)
    if i + 2 >= len(df):
        break

    canal = int(df.iloc[i]["spi"].strip("()"))
    if(canal != 49):
        continue
    byte_alto = int(df.iloc[i+1]["spi"].strip("()"))
    byte_bajo = int(df.iloc[i+2]["spi"].strip("()"))
    tiempo = df.iloc[i]["Time"]

    # Si alguno es None, saltamos este trío
    if any(v is None for v in [canal, byte_alto, byte_bajo]):
        continue

    # Combinar bytes
    valor = (byte_alto << 8) | byte_bajo

    muestras.append((tiempo, valor))

df_muestras = pd.DataFrame(muestras, columns=["tiempo", "valor"])


In [ ]:
import plotly.express as px
import pandas as pd

# Suponiendo que ya tienes tu DataFrame
# df_muestras = pd.DataFrame(muestras, columns=["tiempo", "valor"])

fig = px.line(df_muestras, x="tiempo", y="valor", title="Señal SPI - Canal 49",
              labels={"tiempo": "Tiempo", "valor": "Valor (16 bits)"})

# Mostrar el gráfico interactivo
fig.show()


In [8]:
df_muestras.to_csv("M3_10HZ_DECVbigendian.csv", index=False, sep=';')
